# 트리 (Tree)

## 트리의 부모 찾기

- 문제 출처: [백준 11725번](https://www.acmicpc.net/problem/11725)

`-` 각 노드에 대해 모든 연결 노드를 그래프에 기록한다

`-` 루트 노드부터 BFS를 사용하자

`-` 탐색 과정에서 다음 노드의 부모는 현재 노드가 된다

In [1]:
from collections import defaultdict, deque


def solution():
    N = int(input())
    ROOT = 1
    graph = defaultdict(list)
    visited = {ROOT: True}
    node2parent = {}
    for _ in range(N - 1):
        a, b = map(int, input().split())
        graph[a].append(b)
        graph[b].append(a)
    queue = deque([ROOT])
    while queue:
        node = queue.popleft()
        for next_node in graph[node]:
            if next_node not in visited:
                visited[next_node] = True
                node2parent[next_node] = node
                queue.append(next_node)
    for node in range(2, N + 1):
        print(node2parent[node])


solution()

# input
# 12
# 1 2
# 1 3
# 2 4
# 3 5
# 3 6
# 4 7
# 4 8
# 5 9
# 5 10
# 6 11
# 6 12

 12
 1 2
 1 3
 2 4
 3 5
 3 6
 4 7
 4 8
 5 9
 5 10
 6 11
 6 12


1
1
2
3
3
4
4
5
5
6
6


## 트리 순회

- 문제 출처: [백준 1991번](https://www.acmicpc.net/problem/1991)

`-` 전형적인 트리 순회 문제

`-` 클래스와 재귀 함수를 이용해 깔끔하게 구현할 수 있다

`-` 내 코드보다 좋은 코드가 많이 있음 (배우자)

In [2]:
from collections import defaultdict


def preorder_traversal(tree, start, visited={}):
    visited[start] = True
    for next_node in tree[start]:
        if next_node != ABSENCE and next_node not in visited:
            preorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def inorder_traversal(tree, start, visited={}):
    left = tree[start][0]
    right = tree[start][1]
    if (left == ABSENCE or left in visited):
        visited[start] = True
    for next_node in [left, start, right]:
        if next_node != ABSENCE and next_node not in visited:
            inorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def postorder_traversal(tree, start, visited={}):
    left = tree[start][0]
    right = tree[start][1]
    if (left == ABSENCE or left in visited) and (right == ABSENCE or right in visited):
        visited[start] = True
    for next_node in [left, right, start]:
        if next_node != ABSENCE and next_node not in visited:
            postorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def solution():
    global ABSENCE
    N = int(input())
    tree = defaultdict(list)
    ROOT = "A"
    ABSENCE = "."
    for _ in range(N):
        parent, left, right = input().split()
        tree[parent].append(left)
        tree[parent].append(right)
    preorder_result = preorder_traversal(tree, ROOT)
    inorder_result = inorder_traversal(tree, ROOT)
    postorder_result = postorder_traversal(tree, ROOT)
    for result in [preorder_result, inorder_result, postorder_result]:
        print("".join(result))


solution()

# input
# 7
# A B C
# B D .
# C E F
# E . .
# F . G
# D . .
# G . .

 7
 A B C
 B D .
 C E F
 E . .
 F . G
 D . .
 G . .


ABDCEFG
DBAECFG
DBEGFCA
